# Data Understanding

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 100)
import numpy as np

from fuzzywuzzy import fuzz, process

## Import Data

### Corona

In [2]:
# Read current corona cases
df = pd.read_csv("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_daily_reports/03-14-2020.csv")

In [3]:
df.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Hubei,China,2020-03-14T10:13:09,67790,3075,52960,30.9756,112.2707
1,NaN,Italy,2020-03-14T20:13:16,21157,1441,1966,41.8719,12.5674
2,NaN,Iran,2020-03-14T11:33:06,12729,611,2959,32.4279,53.6880
3,NaN,"Korea, South",2020-03-14T01:33:02,8086,72,510,35.9078,127.7669
4,NaN,Spain,2020-03-14T22:13:11,6391,195,517,40.4637,-3.7492


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 8 columns):
Province/State    114 non-null object
Country/Region    249 non-null object
Last Update       249 non-null object
Confirmed         249 non-null int64
Deaths            249 non-null int64
Recovered         249 non-null int64
Latitude          249 non-null float64
Longitude         249 non-null float64
dtypes: float64(2), int64(3), object(3)
memory usage: 15.7+ KB


In [260]:
# Country dict to match population data
country_dict = {'Mainland China': 'China',
                'Korea, South': 'Republic of Korea',
                'US':'United States of America',
                'Taiwan*': "China, Taiwan Province of China",
                'Bolivia': 'Bolivia (Plurinational State of)',
                "Iran": "Iran (Islamic Republic of)",
                "Russia": "Russian Federation",
                "Vietnam": "Viet Nam",
                "Brunei":"Brunei Darussalam",
                "Moldova": "Republic of Moldova",
                "Cote d'Ivoire": "Côte d'Ivoire",
                "Reunion": "Réunion",
                "Congo (Kinshasa)":"Democratic Republic of the Congo",
                "occupied Palestinian territory": "State of Palestine",
                "Curacao":"Curaçao",
                "Venezuela":"Venezuela (Bolivarian Republic of)",
                #"Jersey": "United Kingdom",
                #"Guernsey": "United Kingdom"
               }
# Replacing names of countries eg. "Mainland China" with "China" 
df['Country'] = df['Country/Region'].replace(country_dict)

### Population

In [6]:
# Read population data
pop = pd.read_excel("data/pop/un_pop_clean.xlsx", index_col=0)
# pop = pop.rename(columns={'2018':'population', "Country Name":"Country"})

# Change Population from thousands to ones
pop.iloc[:,-1] = pop.Population * 1000

In [280]:
pop.head()

,Region,Country code,Type,Parent code,Population
1,WORLD,900,World,0,7.794799e+09
3,More developed regions,901,Development Group,1803,1.273304e+09
4,Less developed regions,902,Development Group,1803,6.521494e+09
5,Least developed countries,941,Development Group,902,1.057438e+09
6,"Less developed regions, excluding least develo...",934,Development Group,902,5.464056e+09


In [44]:
pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 285 entries, 1 to 289
Data columns (total 5 columns):
Region          285 non-null object
Country code    285 non-null int64
Type            285 non-null object
Parent code     285 non-null int64
Population      285 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 23.4+ KB


### Location

In [63]:
# Read location data to get Income levels and additional info
location = pd.read_excel("data/pop/un_location_clean.xlsx", index_col=0)

# Select only countries, not region
#location = location.loc[location.LocTypeName == 'Country/Area'].iloc[:, [0, 2, 3]].reset_index(drop=True)

In [64]:
location.head()

,Location,Notes,LocID,ISO3_Code,LocType,LocTypeName,ParentID,WorldID,SubRegID,SubRegName,...,WB_LMIC,WB_LIC,WB_NoIncomeGroup,MaxHIV_Male,MaxHIV_Female,MaxHIV_BothSexes,YearMaxHIV_BothSexes,HIVAIDSMortalityImpact_AgePattern,HIVAIDSMortalityImpact_e0,TotPop2019LessThan90k
Index,,,,,,,,,,,,,,,,,,,,,
1,WORLD,NaN,900,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,UN development groups,a,1803,NaN,25.0,Label/Separator,900,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,More developed regions,b,901,NaN,5.0,Development group,1803,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Less developed regions,c,902,NaN,5.0,Development group,1803,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Least developed countries,d,941,NaN,5.0,Development group,902,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
group = location[location.LocTypeName.isin(['Development group', 'Income group'])][['Location', 'LocID', 'LocTypeName']].reset_index(drop=True)
group = group.rename(columns={'Location': 'Income'})

In [66]:
location['WD'] = location.loc[:, location.columns.str.startswith("WB")].min(axis=1)

In [67]:
location = location.merge(group, how='left', left_on='WD', right_on = 'LocID', suffixes=('', 'y_'))

### Country names

In [92]:
# Read country names for matching
country = pd.read_csv("data/pop/WDICountry.csv").iloc[:,:10]

# Drop regions or special areas such as OECD countries. Only keep countries
country = country.loc[~country.Region.isna()]

In [107]:
country.sample(5)

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code
177,NZL,New Zealand,New Zealand,New Zealand,NZ,New Zealand dollar,Fiscal year end: March 31; reporting period fo...,East Asia & Pacific,High income,NZ
239,TTO,Trinidad and Tobago,Trinidad and Tobago,Republic of Trinidad and Tobago,TT,Trinidad and Tobago dollar,NaN,Latin America & Caribbean,High income,TT
65,EGY,Egypt,"Egypt, Arab Rep.",Arab Republic of Egypt,EG,Egyptian pound,NaN,Middle East & North Africa,Lower middle income,EG
9,ASM,American Samoa,American Samoa,American Samoa,AS,U.S. dollar,NaN,East Asia & Pacific,Upper middle income,AS
249,UZB,Uzbekistan,Uzbekistan,Republic of Uzbekistan,UZ,Uzbek sum,NaN,Europe & Central Asia,Lower middle income,UZ


In [94]:
country.shape

(217, 10)

In [216]:
location = location.merge(country.iloc[:, [0,1]], left_on='ISO3_Code', right_on='Country Code', how='left')
location = location.iloc[:, [0,1,2,4]]

In [217]:
location.head()

,Location,LocID,ISO3_Code,Country Code
0,Burundi,108,BDI,BDI
1,Comoros,174,COM,COM
2,Djibouti,262,DJI,DJI
3,Eritrea,232,ERI,ERI
4,Ethiopia,231,ETH,ETH


In [156]:
location.isna().sum()

Location         0
LocID            0
ISO3_Code        0
Country Code    19
Short Name      19
dtype: int64

In [162]:
location[location['Short Name'].isna()]

,Location,LocID,ISO3_Code,Short Name
9,Mayotte,175,MYT,NaN
11,Réunion,638,REU,NaN
47,Saint Helena,654,SHN,NaN
57,Western Sahara,732,ESH,NaN
93,"China, Taiwan Province of China",158,TWN,NaN
109,Anguilla,660,AIA,NaN
114,"Bonaire, Sint Eustatius and Saba",535,BES,NaN
122,Guadeloupe,312,GLP,NaN
125,Martinique,474,MTQ,NaN
126,Montserrat,500,MSR,NaN


### Merge Location and Population

In [68]:
# Merge with location data to include Country ISO3 Code
pop = pop.merge(location, left_on="Country code", right_on="LocID")
pop = pop[['Region', 'Population', 'ISO3_Code', 'Income']]

In [69]:
pop.head()

,Region,Population,ISO3_Code,Income
0,WORLD,7.794799e+09,NaN,NaN
1,More developed regions,1.273304e+09,NaN,NaN
2,Less developed regions,6.521494e+09,NaN,NaN
3,Least developed countries,1.057438e+09,NaN,NaN
4,"Less developed regions, excluding least develo...",5.464056e+09,NaN,NaN


In [71]:
# Save final population data
pop.to_pickle('data/pop/pop.pkl')

## Match

In [ ]:
scorer_dict = { 'R':fuzz.ratio,  # Good
                'PR': fuzz.partial_ratio,  # So so
                'TSeR': fuzz.token_set_ratio,  # Good ***
                'TSoR': fuzz.token_sort_ratio,  # Good ***
                'PTSeR': fuzz.partial_token_set_ratio,  # Don't use
                'PTSoR': fuzz.partial_token_sort_ratio, # Don't use
                'WR': fuzz.WRatio,  # Don't use
                'QR': fuzz.QRatio,  # Good
                'UWR': fuzz.UWRatio,  # Don't use
                'UQR': fuzz.UQRatio }  # Good

In [73]:
%%time

def match_country(query, choices, scorer=fuzz.partial_token_sort_ratio, score_cutoff=80):

    match = process.extractOne(query=query, choices=choices, scorer=scorer, score_cutoff=score_cutoff)
    if match:
        return query, match[0], match[1], match[2]
    else:
        return (np.nan, np.nan, np.nan, np.nan)

    
result = df['Country/Region'].drop_duplicates().apply(match_country, choices=pop.Region,  
                                              scorer=fuzz.QRatio, score_cutoff=0)

result = pd.DataFrame(result.tolist(), columns=['country', 'matched_country', 'score', 'key'])

CPU times: user 365 ms, sys: 4.36 ms, total: 370 ms
Wall time: 387 ms


In [ ]:
def merge_match(df, result, fonds_column="fonds_clean",ISIN_column="ISIN", result_isin="ISIN", result_fonds="fonds"):
    '''Fill some missing ISIN values with matching the fund names.'''

    # Create a dictionary from current fond-ISIN pairs
    match_dict = dict(zip(result[result_fonds], result[result_isin]))

    # Fill ISIN column with mapped values
    df[ISIN_column] = df[ISIN_column].fillna(df[fonds_column].map(match_dict))
    
    return df

# Merge the previously matched result with the df
df = preprocessing.merge_match(df, result[result.score>70])

## Merge

In [292]:
# Merge COVID data with population data
merged = df.merge(pop, how="left", left_on="Country", right_on="Region")
merged.drop('Region', axis=1, inplace=True)

In [300]:
merged.sample(5)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,Country,Population,ISO3_Code
228,NaN,Guyana,2020-03-11T20:00:00,1,1,0,5.000000,-58.750000,Guyana,7.865590e+05,GUY
205,North Dakota,US,2020-03-11T20:00:00,1,0,0,47.528912,-99.784012,United States of America,3.310026e+08,USA
27,Inner Mongolia,China,2020-03-11T03:53:03,75,1,71,44.093500,113.944800,China,1.439324e+09,CHN
26,Xinjiang,China,2020-03-11T02:18:14,76,3,73,41.112900,85.240100,China,1.439324e+09,CHN
172,Alaska,US,2020-03-11T20:00:00,1,0,0,61.370716,-152.404419,United States of America,3.310026e+08,USA


In [294]:
merged.isna().sum()

Province/State    117
Country/Region      0
Last Update         0
Confirmed           0
Deaths              0
Recovered           0
Latitude            0
Longitude           0
Country             0
Region              1
Population          1
ISO3_Code           1
dtype: int64

In [301]:
merged[merged.Population.isna()]

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude,Country,Population,ISO3_Code
60,Diamond Princess,Cruise Ship,2020-03-11T20:00:00,696,7,325,35.4498,139.6649,Cruise Ship,NaN,NaN


In [302]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 0 to 229
Data columns (total 11 columns):
Province/State    113 non-null object
Country/Region    230 non-null object
Last Update       230 non-null object
Confirmed         230 non-null int64
Deaths            230 non-null int64
Recovered         230 non-null int64
Latitude          230 non-null float64
Longitude         230 non-null float64
Country           230 non-null object
Population        229 non-null float64
ISO3_Code         229 non-null object
dtypes: float64(3), int64(3), object(5)
memory usage: 21.6+ KB


## Add Extra Columns

In [321]:
grouped = df.groupby("Country/Region", as_index=False)[['Confirmed', 'Deaths','Recovered']].sum()

In [322]:
grouped.head()

,Country/Region,Confirmed,Deaths,Recovered
0,Afghanistan,7,0,0
1,Albania,33,1,0
2,Algeria,26,2,8
3,Andorra,1,0,0
4,Antigua and Barbuda,1,0,0


In [312]:
# Merge with population
agg = grouped.merge(pop, how="left", left_on="Country/Region", right_on="Region")

In [313]:
agg.head()

,Country/Region,Confirmed,Deaths,Recovered,Region,Population,ISO3_Code
0,Afghanistan,7,0,0,Afghanistan,38928341.0,AFG
1,Albania,33,1,0,Albania,2877800.0,ALB
2,Algeria,26,2,8,Algeria,43851043.0,DZA
3,Andorra,1,0,0,Andorra,77265.0,AND
4,Antigua and Barbuda,1,0,0,Antigua and Barbuda,97928.0,ATG


In [314]:
# Create "Active" column
agg['Active'] = agg.Confirmed - (agg.Deaths + agg.Recovered)

In [316]:
# Create "per Capita" columns
agg['Confirmed_per_Cap'] = agg.Confirmed/ agg.Population
agg['Deaths_per_Cap'] = agg.Deaths/ agg.Population
agg['Recovered_per_Cap'] = agg.Recovered/ agg.Population
agg['Active_per_Cap'] = agg.Active/ agg.Population

In [317]:
agg.head()

,Country/Region,Confirmed,Deaths,Recovered,Region,Population,ISO3_Code,Active,Confirmed_per_Cap,Deaths_per_Cap,Recovered_per_Cap,Active_per_Cap
0,Afghanistan,7,0,0,Afghanistan,38928341.0,AFG,7,1.798176e-07,0.000000e+00,0.000000e+00,1.798176e-07
1,Albania,33,1,0,Albania,2877800.0,ALB,32,1.146709e-05,3.474877e-07,0.000000e+00,1.111961e-05
2,Algeria,26,2,8,Algeria,43851043.0,DZA,16,5.929163e-07,4.560895e-08,1.824358e-07,3.648716e-07
3,Andorra,1,0,0,Andorra,77265.0,AND,1,1.294247e-05,0.000000e+00,0.000000e+00,1.294247e-05
4,Antigua and Barbuda,1,0,0,Antigua and Barbuda,97928.0,ATG,1,1.021158e-05,0.000000e+00,0.000000e+00,1.021158e-05


In [68]:
# Save merged DataFrame
agg.to_excel("data/corona with population.xlsx")